<a href="https://colab.research.google.com/github/mspotanski/CSCI5527_Project/blob/main/code/finetune_resnet_ImageNette.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning ResNet on CIFAR10 dataset

#### Set-up git environment

In [1]:
# Clone repository in Google Colab: only needs to be done once
# NOTE: If you have already done this, comment out all code EXCEPT cd command
# from google.colab import auth
# auth.authenticate_user()

# # Set up Git (only needed once per session)
# !git config --global user.email "your_github_email@email.com"
# !git config --global user.name "your_github_username"

# # Clone the repo:
# !git clone https://github.com/mspotanski/CSCI5527_Project.git
# %cd CSCI5527_Project

### Fine-Tuning

In [2]:
import torch
from torchvision import models, datasets, transforms
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

In [3]:
# Set device
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [5]:
# Load flowers dataset and ResNet model
# Normalize current dataset to specifics of original ImageNet dataset to stabilize and speed up learning
# Reseize images to match previous implementation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225])
])

# Load dataset
# Want entire dataset, so need to do 2 different calls
train = datasets.Imagenette(root='./data', split='train', download=True,
                            transform=transform)
test = datasets.Imagenette(root='./data', split="val", download=True,
                           transform=transform)

# Load ResNet model
# For scalability, we will only use the 18-layer version as its the smallest
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

# Set up data pipelines for train,test, and val datasets
train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=64, shuffle=True)

# Modify ResNet Classifier for the 10 classes from the dataset
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)
model = model.to(device)

100%|██████████| 1.56G/1.56G [00:32<00:00, 48.3MB/s]
Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 159MB/s]


In [6]:
# Define opt, loss function, and hyperparameters
loss_rate = 1e-4
epochs = 5
loss_crit = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=loss_rate)

In [7]:
# Fine-Tune ResNet18 on Flowers102 dataset with validation phase
for epoch in range(epochs):
    # Start training phase
    model.train()
    print(f"Current Epoch: {epoch}")
    # Loop through images in train set
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        opt.zero_grad()

        # Make prediction at current step and evaluate loss
        pred = model(images)
        loss = loss_crit(pred, labels)

        # Backpropogate and update model weights
        loss.backward()
        opt.step()


Current Epoch: 0
Current Epoch: 1
Current Epoch: 2
Current Epoch: 3
Current Epoch: 4


In [8]:
# Evaluate Accuracy of Model
correct, total = 0, 0
model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        pred = model(images)
        _, predicted = pred.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# print and store accuracy value
test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Test Accuracy: 97.61%


In [10]:
# Save fine-tuned model and weights
torch.save(model.state_dict(), './resnet18_ImageNette.pth')

# NOTE: this saves model to the google colab local storage
# be sure to download the .pth file, and upload to GitHub directly